
# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 1차시(24.11.21)

---


In [69]:
import os
import shutil
import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [70]:
lines = pd.read_csv('./data/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
lines.head()

,src,tar,lic
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [71]:
del lines['lic']

In [72]:
print(f'전체 데이터 갯수 : {len(lines)}')

전체 데이터 갯수 : 232736


In [73]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [74]:
lines = lines[:60000]
# 데이터 개수 줄이기
lines.sample(10)

,src,tar
36984,Tom abandoned them.,Tom les a abandonnées.
50182,I'm in love with you.,Je suis amoureux de toi.
15018,He likes to run.,Il aime courir.
1369,Keep them.,Gardez-les.
49138,I like to give gifts.,J'aime donner des cadeaux.
36617,They can't do this.,Elles ne peuvent faire cela.
47219,Do you know his name?,Connaissez-vous son nom ?
51537,She has a funny face.,Elle a un drôle de visage.
41221,I know Tom is tough.,Je sais que Tom est sévère.
29650,They were cousins.,Elles étaient cousines.


In [75]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n') # 공백으로 구분을 지어줘야 한다
# 시작을 의미하는 sos, 끝을 의미하는 eos(특수토큰)
# 실행은 한번만 여러번 하면 여러개가 붙는다

In [76]:
lines.sample(10)

,src,tar
49357,I really love French.,\t J'aime beaucoup le français. \n
28714,No one can get in.,\t Personne ne peut s'y introduire. \n
28290,It won't work out.,\t Ça ne marchera pas. \n
26596,I can't place him.,\t Je ne parviens pas à le remettre. \n
22811,Thanks very much.,\t Mille mercis. \n
35056,Is that deliberate?,\t Est-ce délibéré ? \n
47018,Damascus is in Syria.,\t Damas est en Syrie. \n
12101,Is Tom working?,\t Est-ce que Tom travaille ? \n
59705,No one knew who I was.,\t Personne ne savait qui j'étais. \n
28561,Let's remain calm.,\t Restons calmes. \n


In [77]:
src_vocab = set() # 비어있는 집합 생성
for line in lines['src']:
    for char in line:
        src_vocab.add(char)
        # src 문자 집합 구축

In [78]:
print(src_vocab)

{"'", '0', 'Q', 'f', '?', '5', 'z', ',', '!', 'y', 'l', 'S', 'L', 'p', '€', '2', 'r', 'P', 'X', 'd', 'j', 'ï', 'C', '1', 'H', 'i', 'q', 'N', 'v', 'F', 'Y', 'k', 'M', 'n', '7', 'I', '$', ' ', 'w', 'J', 'O', 'c', '6', 'K', '9', 'E', '"', 'R', 'é', 'u', '%', '&', 't', '4', ':', 'U', 'V', 'g', 'b', 'e', 'B', 'm', 's', '-', 'G', 'h', 'A', 'Z', '’', 'W', '/', 'o', 'a', '3', 'x', 'D', 'T', '.', '8'}


In [79]:
tar_vocab = set()
for line in lines['tar']:
    for char in line:
        tar_vocab.add(char)
        # tar 문자 집합 구축

In [80]:
print(tar_vocab)

{"'", '0', 'Q', 'f', 'Ê', '?', '5', 'z', ',', '!', 'œ', 'y', 'l', 'S', 'L', '\u2009', 'p', '2', 'r', 'P', 'X', 'j', 'à', 'd', 'ï', 'C', '1', 'H', 'i', 'q', 'N', 'v', 'F', 'Y', 'û', 'k', '‽', '\u202f', 'M', '\t', 'n', '7', 'I', '$', '»', ' ', 'w', 'J', 'O', 'c', '6', 'K', '9', 'E', '"', 'R', 'ô', 'Ç', 'é', 'u', '%', 'ê', '&', 't', '4', 'ç', 'î', ':', 'U', 'V', 'è', 'g', 'b', 'e', '«', 'B', '\xa0', 's', 'm', 'Ô', '-', 'G', 'h', 'A', '‘', 'É', '’', 'W', 'o', 'a', '3', 'x', 'D', 'ë', 'T', '.', 'ù', '\n', 'À', 'â', '8'}


In [81]:
src_vocab_size = len(src_vocab) +1
print(f'src 문장의 문자 집합 크기 : {src_vocab_size}')
# 인코더에 데이터를 넣을 때 사용될 차원의 크기

src 문장의 문자 집합 크기 : 80


In [82]:
tar_vocab_size = len(tar_vocab) +1
print(f'tar 문장의 문자 집합 크기 : {tar_vocab_size}')
# 디코더에 데이터를 넣을때 사용될 차원의 크기

tar 문장의 문자 집합 크기 : 102


In [83]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€'}

In [84]:
src_to_index = {} # 인덱스를 담아줄 빈 딕셔너리

for i, word in enumerate(src_vocab):
    src_to_index[word] = i + 1
# 정수 인덱싱을 위한 인코딩 작업
# 패딩을 위한 +1

In [85]:
print(src_to_index)

{"'": 1, '0': 2, 'Q': 3, 'f': 4, '?': 5, '5': 6, 'z': 7, ',': 8, '!': 9, 'y': 10, 'l': 11, 'S': 12, 'L': 13, 'p': 14, '€': 15, '2': 16, 'r': 17, 'P': 18, 'X': 19, 'd': 20, 'j': 21, 'ï': 22, 'C': 23, '1': 24, 'H': 25, 'i': 26, 'q': 27, 'N': 28, 'v': 29, 'F': 30, 'Y': 31, 'k': 32, 'M': 33, 'n': 34, '7': 35, 'I': 36, '$': 37, ' ': 38, 'w': 39, 'J': 40, 'O': 41, 'c': 42, '6': 43, 'K': 44, '9': 45, 'E': 46, '"': 47, 'R': 48, 'é': 49, 'u': 50, '%': 51, '&': 52, 't': 53, '4': 54, ':': 55, 'U': 56, 'V': 57, 'g': 58, 'b': 59, 'e': 60, 'B': 61, 'm': 62, 's': 63, '-': 64, 'G': 65, 'h': 66, 'A': 67, 'Z': 68, '’': 69, 'W': 70, '/': 71, 'o': 72, 'a': 73, '3': 74, 'x': 75, 'D': 76, 'T': 77, '.': 78, '8': 79}


In [86]:
tar_to_index = {}
for i, word in enumerate(tar_vocab):
    tar_to_index[word] = i + 1

In [87]:
print(tar_to_index)

{"'": 1, '0': 2, 'Q': 3, 'f': 4, 'Ê': 5, '?': 6, '5': 7, 'z': 8, ',': 9, '!': 10, 'œ': 11, 'y': 12, 'l': 13, 'S': 14, 'L': 15, '\u2009': 16, 'p': 17, '2': 18, 'r': 19, 'P': 20, 'X': 21, 'j': 22, 'à': 23, 'd': 24, 'ï': 25, 'C': 26, '1': 27, 'H': 28, 'i': 29, 'q': 30, 'N': 31, 'v': 32, 'F': 33, 'Y': 34, 'û': 35, 'k': 36, '‽': 37, '\u202f': 38, 'M': 39, '\t': 40, 'n': 41, '7': 42, 'I': 43, '$': 44, '»': 45, ' ': 46, 'w': 47, 'J': 48, 'O': 49, 'c': 50, '6': 51, 'K': 52, '9': 53, 'E': 54, '"': 55, 'R': 56, 'ô': 57, 'Ç': 58, 'é': 59, 'u': 60, '%': 61, 'ê': 62, '&': 63, 't': 64, '4': 65, 'ç': 66, 'î': 67, ':': 68, 'U': 69, 'V': 70, 'è': 71, 'g': 72, 'b': 73, 'e': 74, '«': 75, 'B': 76, '\xa0': 77, 's': 78, 'm': 79, 'Ô': 80, '-': 81, 'G': 82, 'h': 83, 'A': 84, '‘': 85, 'É': 86, '’': 87, 'W': 88, 'o': 89, 'a': 90, '3': 91, 'x': 92, 'D': 93, 'ë': 94, 'T': 95, '.': 96, 'ù': 97, '\n': 98, 'À': 99, 'â': 100, '8': 101}


In [88]:
encoder_input = []
for line in lines['src']:
    encoded_line = []
    for char in line:
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)
print(f'src문장의 정수 인코딩 {encoder_input[:5]}')
# 영어

src문장의 정수 인코딩 [[65, 72, 78], [65, 72, 78], [65, 72, 78], [65, 72, 78], [25, 26, 78]]


In [ ]:
decoder_input = []
for line in lines['tar']:
    decoded_line = []
    for char in line:
        decoded_line.append(tar_to_index[char])
    decoder_input.append(decoded_line)
print(f'tar문장의 정수 인코딩 {decoder_input[:5]}')
# 프랑스어

tar문장의 정수 인코딩 [[40, 46, 70, 90, 46, 10, 46, 98], [40, 46, 39, 90, 19, 50, 83, 74, 96, 46, 98], [40, 46, 54, 41, 46, 19, 89, 60, 64, 74, 46, 10, 46, 98], [40, 46, 76, 89, 60, 72, 74, 46, 10, 46, 98], [40, 46, 14, 90, 13, 60, 64, 46, 10, 46, 98]]


In [90]:
decoder_target = []
for line in lines['tar']:
    char_position = 0 # 초기화
    # 문자 위치를 추적할 수 있는 변수
    decoded_line = []
    for char in line: # 문장의 각 낱글자 char
        if char_position != 0: # sos를 제외 하기 위해서 사용
            decoded_line.append(tar_to_index[char])
        char_position = char_position + 1
    decoder_target.append(decoded_line)
print(f'디코더 target 문장의 정수 인코딩 {decoder_target[:5]}')

디코더 target 문장의 정수 인코딩 [[46, 70, 90, 46, 10, 46, 98], [46, 39, 90, 19, 50, 83, 74, 96, 46, 98], [46, 54, 41, 46, 19, 89, 60, 64, 74, 46, 10, 46, 98], [46, 76, 89, 60, 72, 74, 46, 10, 46, 98], [46, 14, 90, 13, 60, 64, 46, 10, 46, 98]]


In [91]:
lengths = []
for line in lines['src']:
    lengths.append(len(line))
max_src_len = max(lengths)
print(f'src 문장의 최대 길이 : {max_src_len}')

src 문장의 최대 길이 : 22


In [92]:
lengths = []
for line in lines['tar']:
    lengths.append(len(line))
max_tar_len = max(lengths)
print(f'tar 문장의 최대 길이 : {max_tar_len}')

tar 문장의 최대 길이 : 76


In [93]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post') # 문장의 시퀀스 순서자체가 중요할때는 패딩을 뒤에 다는 경우가 많다
# 인코더에 들어갈 데이터 패딩 작업

In [94]:
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
# 디코더에 들어갈 데이터 패딩 작업

In [95]:
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
# 디코더의 정답 데이터로 사용될 데이터 패딩 작업

# 1. 인코더와 디코더의 문장 길이는 동일하게 맞추지 않아도 된다
# - 인코더 데이터는 인코더 데이터 끼리, 디코더 데이터는 디코더 데이터 끼리 맞추어 패딩하면 딘다
# 2. 패딩을 앞에 둘까, 뒤에 둘까?
# - 문장 생성 시에는 문장 끝에 어떤 단어가 나올지가 더 중요하기 때문에 패딩을 앞에 두고
# - 문장의 순서, 시퀀스를 유지하는게 더 중요한 경우에는 패딩을 뒤쪽에 둔다
# 3. 디코더의 target데이터에서는 <sos>가 제거 되었는데 동일하게 패딩을 주어도 되는가?
# - 패딩은 부족한 부분을 0을 채워 동일한 길이로 맞추는 과정이기 때문에, target과 decoder의 길이 차이는 중요하지 않다.

In [96]:
# one-hot vector
encoder_input = to_categorical(encoder_input)

In [97]:
decoder_input = to_categorical(decoder_input)

In [98]:
decoder_target = to_categorical(decoder_target)
# 여기까지가 데이터 전처리

In [99]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [100]:
encoder_inputs = Input(shape = (None, src_vocab_size))
# (None, src_vocab_size)의 형태를 갖는 인붓을 주겠다
# None : 가변적인 입력 문장의 길이 (즉, 따로 명시하지 않겠다라는 의미)
# abc, de
# [[0,0,1], [0,1,0],[1,0,0]] # abc
# [[1,0,1],[1,1,1]] # de

In [101]:
encoder_lstm = LSTM(units=256, return_state=True)

In [102]:
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# LSTM은 은닉 상태(state_h)와 셀(state_c) 상태를 리턴한다!!

In [103]:
encoder_states = [state_h, state_c]
# context vector

In [104]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
# 디코더는 인코더의 마지막 context vector를 초기 은닉 상태로 사용한다.

In [105]:
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
# return_sequences : 모든 타임스텝의 출력
# return_state : 마지막 타임스텝의 은닉상태와 셀상태를 출력

In [106]:
# 교사 강요의 시작
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 ouputs 출력되는 구간

In [107]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
# 디코더의 출력은 타겟 단어 집합의 각 단어에 대한 확률 중 높은 값이어야 한다.

In [108]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [109]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = 'rmsprop', loss= 'categorical_crossentropy')

In [ ]:
model.fit(
    x = [encoder_input, decoder_input],
    # 모델에 입력할 데이터
    y = decoder_target, 
    batch_size = 64,
    epochs = 10, # 잘 나올려면 15 ~ 20 정도 줘야 되지만 시간이슈
    validation_split = 0.2 
)
# ETA가 학습에 걸리는 시간 표시

Epoch 1/10
750/750 [==============================] - 228s 301ms/step - loss: 0.7208 - val_loss: 0.6331
Epoch 2/10
750/750 [==============================] - 217s 290ms/step - loss: 0.4412 - val_loss: 0.5127
Epoch 3/10
750/750 [==============================] - 256s 341ms/step - loss: 0.3696 - val_loss: 0.4487
Epoch 4/10
750/750 [==============================] - 237s 316ms/step - loss: 0.3291 - val_loss: 0.4143
Epoch 5/10
750/750 [==============================] - 261s 348ms/step - loss: 0.3013 - val_loss: 0.3909
Epoch 6/10
750/750 [==============================] - 232s 309ms/step - loss: 0.2811 - val_loss: 0.3740
Epoch 7/10
750/750 [==============================] - 227s 303ms/step - loss: 0.2654 - val_loss: 0.3634
Epoch 8/10
750/750 [==============================] - 237s 316ms/step - loss: 0.2534 - val_loss: 0.3551
Epoch 9/10
750/750 [==============================] - 215s 287ms/step - loss: 0.2424 - val_loss: 0.3485
Epoch 10/10
750/750 [==============================] - 214s 285m

In [146]:
# 번역 동작 step
# 1. 번역하려고 사는 input 문장이 인코더에 들어가서 context벡터를 얻는다
# 2. context벡터와 <sos> (\t)를 디코더로 보낸다
# 3. 디코더가 <eos> (\n)이 나올 때 까지 다음 문자를 에측 반복

encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)

In [147]:
decoder_state_input_h = Input(shape=(256, )) # 은닉상태 # 위의 units의 크기와 맞춰서 256
decoder_state_input_c = Input(shape=(256, )) # 셀상태

# 디코더 셀에서 각각 이전 시점의 상태를 저장하는 텐서
# 디코더의 은닉 상태와 셀 상태를 입력으로 받기 위한 텐서로, 디코더 LSTM의 hidden units 크기와 동일하게 넣어주기

In [148]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [149]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 디코더 lstm모델이 입력값인 decoder)inputs와, 이전 상태인 decoder_states_inputs을 입력 받아
# 다음 단어를 예측하는 출력(decoder_outputs)와 새로운 은닉상태/셀 상태(state_h, state_c)를 계산

In [150]:
decoder_states = [state_h, state_c]

In [151]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
# 디코더 출력값을 소프트맥스 레이어로 통과시켜 단어별 확률 분포 변환

In [152]:
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
# 다음 단어의 확률 분포와 새로운 상태값을 반환

In [153]:
index_to_src = {}
for char, i in src_to_index.items():
    index_to_src[i] = char
    # key : 정수 인덱스, value가 문자로 되도록 뒤집어주기

print(index_to_src)


{1: "'", 2: '0', 3: 'Q', 4: 'f', 5: '?', 6: '5', 7: 'z', 8: ',', 9: '!', 10: 'y', 11: 'l', 12: 'S', 13: 'L', 14: 'p', 15: '€', 16: '2', 17: 'r', 18: 'P', 19: 'X', 20: 'd', 21: 'j', 22: 'ï', 23: 'C', 24: '1', 25: 'H', 26: 'i', 27: 'q', 28: 'N', 29: 'v', 30: 'F', 31: 'Y', 32: 'k', 33: 'M', 34: 'n', 35: '7', 36: 'I', 37: '$', 38: ' ', 39: 'w', 40: 'J', 41: 'O', 42: 'c', 43: '6', 44: 'K', 45: '9', 46: 'E', 47: '"', 48: 'R', 49: 'é', 50: 'u', 51: '%', 52: '&', 53: 't', 54: '4', 55: ':', 56: 'U', 57: 'V', 58: 'g', 59: 'b', 60: 'e', 61: 'B', 62: 'm', 63: 's', 64: '-', 65: 'G', 66: 'h', 67: 'A', 68: 'Z', 69: '’', 70: 'W', 71: '/', 72: 'o', 73: 'a', 74: '3', 75: 'x', 76: 'D', 77: 'T', 78: '.', 79: '8'}


In [154]:
index_to_tar = {}
for char, i in tar_to_index.items():
    index_to_tar[i] = char
print(index_to_tar)

{1: "'", 2: '0', 3: 'Q', 4: 'f', 5: 'Ê', 6: '?', 7: '5', 8: 'z', 9: ',', 10: '!', 11: 'œ', 12: 'y', 13: 'l', 14: 'S', 15: 'L', 16: '\u2009', 17: 'p', 18: '2', 19: 'r', 20: 'P', 21: 'X', 22: 'j', 23: 'à', 24: 'd', 25: 'ï', 26: 'C', 27: '1', 28: 'H', 29: 'i', 30: 'q', 31: 'N', 32: 'v', 33: 'F', 34: 'Y', 35: 'û', 36: 'k', 37: '‽', 38: '\u202f', 39: 'M', 40: '\t', 41: 'n', 42: '7', 43: 'I', 44: '$', 45: '»', 46: ' ', 47: 'w', 48: 'J', 49: 'O', 50: 'c', 51: '6', 52: 'K', 53: '9', 54: 'E', 55: '"', 56: 'R', 57: 'ô', 58: 'Ç', 59: 'é', 60: 'u', 61: '%', 62: 'ê', 63: '&', 64: 't', 65: '4', 66: 'ç', 67: 'î', 68: ':', 69: 'U', 70: 'V', 71: 'è', 72: 'g', 73: 'b', 74: 'e', 75: '«', 76: 'B', 77: '\xa0', 78: 's', 79: 'm', 80: 'Ô', 81: '-', 82: 'G', 83: 'h', 84: 'A', 85: '‘', 86: 'É', 87: '’', 88: 'W', 89: 'o', 90: 'a', 91: '3', 92: 'x', 93: 'D', 94: 'ë', 95: 'T', 96: '.', 97: 'ù', 98: '\n', 99: 'À', 100: 'â', 101: '8'}


In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, tar_vocab_size))
    # (1,1, tar_vocab_size) 배열 생성 -> (배치크기, 시퀀스 길이, 어휘크기)
    # 즉, 1개의 1 x tar_vocab_size로 구성된 3D 배열을 생성 ex. tar_vocab_size가 5일때 -> (0, 0, 0, 0, 0)
    target_seq[0, 0, tar_to_index['\t']] = 1 # 시작 심볼은 원-핫 패턴으로 지정
    # [0, 0, tar_to_index['\t'] = [베치, 행, 인덱스]
    # 0번째 문장의 0번째 단어의 \t에 해당하는 위치를 1로 설정

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition: # stop_condition이 false인 동안 반복문 실행 (not stop_condition = !stop_condition)
        # 즉, not stop_condition은 not stop_condition이 참이 되기 때문에 => while 참 의 반복문이 성립
        # while a : a가 참인 동안 반복
        # while not a : a가 거짓인 동안 반복

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # target_seq : 현재 디코더의 입력 시퀀스
        # states_value : context vector
        # output_tokens : 다음 단어에 대한 예측 확률 분포, h : 은닉 상태 , c : 셀 상태

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # 예측된 데이터 중에서 현재 타임 스텝의 마지막 값을 불러와서 그 중에 argmax(최대값)을 sample_token_index에 저장
        sampled_char = index_to_tar[sampled_token_index]

        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True
            # eos에 도달하거나 최대 문장 길이를 넘어서면 반복을 중단!!

        target_seq = np.zeros((1, 1, tar_vocab_size))
        # 1, 1, tar_vocab_size의 배열 생성
        # (배치사이즈, 시퀀스 길이, 타겟 집합의 크기)
        target_seq[0, 0, sampled_token_index] = 1
        # 첫번째 배치, 첫번째 타입스텝에서 sampled_token_index에 해당하는 위치를 1로 설정

        states_value = [h, c]
        # 현재 시점의 상태를 다음 시점의 상태로 전달
    return decoded_sentence

In [159]:
for seq_index in [500, 10000]:
    input_seq = encoder_input[seq_index:seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(f'입력 문장 : {lines.src[seq_index]}')
    print(f'정답 문장 : {lines.tar[seq_index][2:len(lines.tar[seq_index])-1]}')
    print(f'번역 문장 : {decoded_sentence[1:len(decoded_sentence)]}')

1/1 [==============================] - 0s 29ms/step
입력 문장 : Be brief.
정답 문장 : Soyez brefs. 
번역 문장 : Soyez prudentes ! 

1/1 [==============================] - 0s 22ms/step
입력 문장 : We're so busy.
정답 문장 : On est tellement occupés ! 
번역 문장 : Nous sommes très heureux. 

